In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def load_df(path, encoding="utf-8"):
    loaded_pd = pd.read_csv(path, sep="\t", encoding=encoding)
    pd_name = path.split("/")[-1].split(".")[0]
    return pd_name, loaded_pd

In [ ]:
paths_expression = glob.glob("data/expression data/*.txt")

In [ ]:
expression_data = []
for path in paths_expression:
    expression_data.append(load_df(path))

In [ ]:
paths_other = glob.glob("data/*.txt")
aligner_path = "data/phenotypes_id_aligner.txt"
other_data = []
for path in paths_other:
    if path == aligner_path:
        other_data.append(load_df(path, encoding="latin"))
    else:
        other_data.append(load_df(path))

## We have loaded the dataframes to lists

Each list contains tuples (name, dataframe)
- expression_data: tables with gene expressions
- other_data: all the remaining tables

In [ ]:
# Compute the total set of mice strains present in the expressions dataframes
mice_set = set()

for i in range(len(expression_data)):
    df = expression_data[i][1].set_index('gene')
    mice_set.update(list(df.columns))

# Compute the total list of mice strains sorted alphabetically
mice_list = sorted(list(mice_set))
print(len(mice_list))

In [ ]:
# Create a list of dataframes that always contain all mice (with NaN values if the mice are not originally present)
df_list = []

for i in range(len(expression_data)):
    df = expression_data[i][1].set_index('gene')
    # standardize to fight the issue of different means and deviations - to make all of the values comparable
    df = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)
    missing_mice = mice_set - set(df.columns)
    for mouse in missing_mice:
        df[mouse] = np.NaN
        
    # Order the columns alphabetically
    df = df[mice_list]
    df.reset_index(inplace=True)
    df["gene"] = df["gene"].apply(lambda name: name + "_" + expression_data[i][0])
    df.rename(columns={"gene": "strain"}, inplace=True)
    df.set_index("strain", inplace=True)
    df_list.append(df.transpose())

In [ ]:
base_df = df_list[0]
for other_df in df_list[1:]:
    base_df = base_df.join(other_df)

We note that all in all the tissues apart from [9, 14, 16, 17, 22, 26, 27, 30, 31, 32, 34, 35, 37, 38, 39] the values follow normal distribution. To make all of them comparable we standardize them by (SNP, tissue)

In [ ]:
gene_tissue_num = len(base_df.columns)

In [ ]:
# we remove the mice for which we have no expression data whatsoever
base_df.drop(base_df[base_df.isnull().sum(axis=1) == gene_tissue_num].index, inplace=True)

We remove the (SNP, tissue) for which we do not have data for at least 4 mice

In [ ]:
strains_num = len(base_df)

In [ ]:
columns_to_drop = base_df.columns[base_df.isnull().sum(axis=0) > strains_num - 4]

In [ ]:
base_df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
base_df.to_pickle("processed_data/expression_standardized_cleaned.pkl")

In [ ]:
phenotype_present = pd.read_pickle("processed_data/pheno_joined_present.pkl")

In [ ]:
strains_with_pheno = phenotype_present.columns[:-2]

In [ ]:
base_df = base_df.transpose()
base_df.rename(columns={"strain": "SNP"})

In [ ]:
strain_with_pheno = []
for index, row in phenotype_present.iterrows():
    strain_with_pheno.append(row[~row.isnull()].index.tolist()[:-2])

In [ ]:
# 1. select strain with largest number of SNPs

# 2. as testing for all 2^86 subsets of strains that might have common expressions present is computationally infeasible, we employ a simple strategy:
# greedily add strains that reduce the number of common SNPs the least

# for now, to simplify even more, we just replace the missing values with mean

In [ ]:
df_pheno = []
for index, row in phenotype_present.iterrows():
    row = row[~row.isna()]
    row_df = pd.DataFrame(row)
    column_name = row_df.columns[0]
    row_df.rename(columns={column_name: row["Shown_pheno"]}, inplace=True)
    df_pheno.append(base_df.join(row_df, how="inner"))

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split

In [ ]:
pheno_idx_of_interest = [0, 1, 2, 3, -1, -2]
malaria_susceptibility_idx = [-1]

In [ ]:
import random
positive_test_score = 0
num_tests = 20
for random_state in range(num_tests):
    for pheno_idx in malaria_susceptibility_idx:    
        X_train, X_test, y_train, y_test = train_test_split(
            df_pheno[pheno_idx].iloc[:, :-1].fillna(0), df_pheno[pheno_idx].iloc[:, -1].fillna(0), test_size=0.3, random_state=random_state)

        clf = RidgeCV(alphas=[1e-99, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10, 1e11, 1e12]).fit(X_train, y_train)
        if (clf.alpha_ == 1e12 or clf.alpha_ == 1e-5):
            print(clf.alpha_)
            print("Might need to update alpha")
        test_score = clf.score(X_test, y_test)
        if test_score > 0:
            positive_test_score += 1
        print(f"{df_pheno[pheno_idx].columns[-1]}, test+score: {test_score}, number_of_samples: {len(df_pheno[pheno_idx])}")
print(f"percentage of positive R^2 scores: {float(positive_test_score) / num_tests * 100}")

We conclude that The ridge regression classifier does better than it would by always guessing the expected value, so it is a good baseline and the phenotype appears to be somewhat predictible give the gene expression data.

In [ ]:
# we compare ridge regression with expected value error
df_malaria = df_pheno[-1]

In [ ]:
df_malaria.to_pickle("processed_data/malaria_susceptibility.pkl")

In [ ]:
X_train, y_train = df_malaria.iloc[:, :-1].fillna(0), df_malaria.iloc[:, -1].fillna(0)
clf_alpha = RidgeCV(alphas=np.linspace(7000, 7049, 1000), store_cv_values=True).fit(X_train, y_train)
print(f"best alpha: {clf_alpha.alpha_}")

In [ ]:
clf = RidgeCV(alphas=[clf_alpha.alpha_], store_cv_values=True).fit(X_train, y_train)

print(f"{df_malaria.columns[-1]}, number_of_samples: {len(df_malaria)}")
print(f"mean error: {clf.cv_values_.mean()}")
print(f"mean error for completely susceptible mice: {clf.cv_values_[np.where(y_train == 1.0)[0]].mean()}")
print(f"mean error for somewhat resistant mice: {clf.cv_values_[np.where(y_train != 1.0)[0]].mean()}")

In [ ]:
# error distribution by mice susceptibility
plt.scatter(x=y_train, y=clf.cv_values_)
plt.title("Classification error as a function of susceptibility")
plt.ylabel("classification error %")
plt.xlabel("malaria susceptibility %")

We note that the classifier makes the worst mistakes when predicting susceptibility of the most resistant mice.

In [ ]:
plt.hist(clf.coef_)

Most of the weights are very small, meaning most of the expression data is not used. It appears that there are both gene expressions that correlate with susceptibility as those related to resistance: the histogram has negative and positive entries.

SNPs used by classifier to predict susceptibility:

In [ ]:
X_train.columns[np.where(clf.coef_ > 0.000023)[0]]

SNPs used by classifier to predict resistance:

In [ ]:
X_train.columns[np.where(clf.coef_ < -0.0000215)[0]]

It seems that gene expression in bone femur is the strongest indicator

In [ ]:
from scipy.stats import spearmanr
spearman_correlation = spearmanr(X_train, y_train)